In [130]:
###Imports
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import time

#Plot Size
plt.rcParams["figure.figsize"] = (14, 8)

In [131]:
###Load Data
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

### Feature Engineering

In [132]:
#Relevant Categories
cats = [
    #"datetime"
    "season",
    "holiday",
    "workingday",
    "weather",
    "temp",
    "atemp",
    "humidity",
    "windspeed",
    "weekend",
    "hour",
    "day",
    "month",
    "year",
    "yday"
        ]

In [133]:
#Transform date column to datetime format
train["datetime"] = pd.to_datetime(train["datetime"])


In [134]:
#Create column for weekend
train.loc[(train["datetime"].apply(lambda x: x.weekday() > 4)), "weekend"] = 1
train.loc[(train["datetime"].apply(lambda x: x.weekday() <= 4)), "weekend"] = 0


In [2]:
#Create time columns
train["hour"] = train["datetime"].dt.hour
train["day"] = train["datetime"].dt.day
train["month"] = train["datetime"].dt.month
train["year"] = train["datetime"].dt.year
train["yday"] = train["datetime"].dt.dayofyear
train["wday"] = train["datetime"].dt.dayofweek

NameError: name 'train' is not defined

### Data Exploration

In [5]:
#Difference between holiday, weekday and weekend

sns.lineplot(x=train["hour"],y=train["count"],hue=train["holiday"])
plt.show()
sns.lineplot(x=train["hour"],y=train["count"],hue=train["workingday"])
plt.show()
sns.lineplot(x=train["hour"],y=train["count"],hue=train["weekend"])

# daytype = train[(train["holiday"] == 0)
#     & (train["workingday"] == 0)
#                     ]
# daytype

NameError: name 'sns' is not defined

In [137]:
#Overall Average
overall_mean = int(round(train["count"].mean()))
print(f"Average Usage overall: {overall_mean}")

#Average usage on holidays
holiday_mean = int(round(train["count"].loc[(train["holiday"] == 1)].mean()))
print(f"Average Usage on holidays: {holiday_mean}")

noholiday_mean = int(round(train["count"].loc[(train["holiday"] == 0)].mean()))
print(f"Average Usage on non-holidays: {noholiday_mean}")

#Average usage on weekends
weekend_mean = int(round(train["count"].loc[(train["weekend"] == 1)].mean()))
print(f"Average Usage on weekends: {weekend_mean}")

workingday_mean = int(round(train["count"].loc[(train["workingday"] == 1)].mean()))
print(f"Average Usage on working days: {workingday_mean}")

Average Usage overall: 192
Average Usage on holidays: 186
Average Usage on non-holidays: 192
Average Usage on weekends: 189
Average Usage on working days: 193


In [138]:
sns.heatmap(data=train["count"])


IndexError: Inconsistent shape between the condition and the input (got (10886, 1) and (10886,))

In [ ]:
df = train[
            (train["season"] == 1)
            & (train["year"] == 2012)
            #& (train["yday"].isin(range(100,300)))
        ]


#fig, axs = plt.subplots()
#fig = sns.regplot(x="hour",y="count",data=df)
#fig.set(ylim=(-20,900),xlim=(-5,30))



print(df)

#fig2 = sns.regplot(x="hour",y="count",data=df)
#fig2.set(ylim=(-20,900),xlim=(-5,30))

sns.pairplot(df[["casual",
        "registered",
        "temp",
        "humidity",
        "windspeed",
        "holiday"
        ]], palette="husl", kind="reg")

# plt.show()

### Train Test Split

In [ ]:
X = train.drop(["count","casual","registered"], axis=1)
y = train["count"]

X_train, X_test, y_train, y_test = train_test_split(X[cats], y, train_size=0.8, test_size=0.2, random_state=0)

### Train a model

In [ ]:
#
#
# m = LinearRegression()
#
# m.fit(X_train,y_train)
#
# print(m.score(X_train,y_train))